In [7]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from google.colab import files
uploaded = files.upload()

Saving dataset_short2.xlsx to dataset_short2.xlsx


In [19]:
df = pd.read_excel('dataset_short2.xlsx', header=1)


In [20]:
# === 1. Load and clean the dataset ===

file_path = '/content/drive/MyDrive/dataset_short2.xlsx'

df = pd.read_excel('dataset_short2.xlsx')

# 5. Проверяем первые строки таблицы
df.head()



,№ п/п,м2,НЦП,Итог цена,Цены предложений на циане
0,1,14.9,3767000.0,3804670,5.611111e+06
1,2,18.6,625300.0,не состоялся,1.026125e+06
2,3,17.4,2152700.0,не состоялся,2.600000e+06
3,4,12.5,1115000.0,1325000,2.112500e+06
4,5,13.6,1924100.0,не состоялся,2.000000e+06


In [21]:
# Select relevant columns and rename them
df_model = df[['м2', 'НЦП', 'Итог цена', 'Цены предложений на циане']].copy()
df_model.columns = ['area', 'starting_price', 'final_price', 'market_price']

In [22]:
# Remove rows where the final price is not a valid number (e.g., "cancelled")
df_model = df_model[pd.to_numeric(df_model['final_price'], errors='coerce').notnull()]
df_model['final_price'] = df_model['final_price'].astype(float)

<ipython-input-22-cd2e76356e59>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['final_price'] = df_model['final_price'].astype(float)


In [23]:
# === 2. Prepare features and target variable ===
X = df_model[['area', 'starting_price', 'market_price']]
y = df_model['final_price']

In [24]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# === 3. Train XGBoost regression model ===
model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [27]:
# === 4. Predict and evaluate the model ===
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


RMSE: 383073.6958
R² Score: 0.8391
